In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM 
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam
import numpy as np

In [2]:
df = pd.read_parquet("small_tokenized_data.parquet")
IDS = df["input_ids"].tolist()
newIDS=[]
for each in IDS:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    newIDS.append(each)

In [3]:
#get the star count AKA the y from half_data
df = pd.read_parquet("small_tokenized_data.parquet")
IDS = df["stars"].tolist()
starIDS=[]
for each in IDS:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    starIDS.append(each)

In [4]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [5]:
#load data in batches for bert
bertData = torch.utils.data.TensorDataset(torch.tensor(newIDS), torch.tensor(starIDS))

# set up a data loader to get batches from our dataset
bertLoader = torch.utils.data.DataLoader(bertData, batch_size = 16, shuffle = True)

/var/folders/dg/jkq_f9b51f38pf3ddscgfrtm0000gn/T/ipykernel_37337/400604804.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  bertData = torch.utils.data.TensorDataset(torch.tensor(newIDS), torch.tensor(starIDS))


In [7]:
#PyTorch tensor with first 3 reviews from our training set and pass it to it
#this is the BERTs final layer output for each token
i = 0
for batch in bertLoader:
    i++
    print(i)
    y, pooled = bert(batch[0], output_all_encoded_layers=False)


# NEXT THING TO DO
#neural net model to go from pooled (vetors) to labels

[tensor([[  101,  2057,  2179,  ...,     0,     0,     0],
        [  101,  2023,  2173,  ...,     0,     0,     0],
        [  101,  1045,  2293,  ...,     0,     0,     0],
        ...,
        [  101, 16521,  2001,  ...,     0,     0,     0],
        [  101,  2034,  3942,  ...,     0,     0,     0],
        [  101,  2074,  2178,  ...,     0,     0,     0]]), tensor([4, 4, 4, 4, 0, 4, 4, 1, 2, 3, 0, 4, 4, 0, 4, 4])]
[tensor([[  101,  2985,  2026,  ...,     0,     0,     0],
        [  101,  2398,  2091,  ...,     0,     0,     0],
        [  101,  6739, 15812,  ...,     0,     0,     0],
        ...,
        [  101,  1045,  2228,  ...,     0,     0,     0],
        [  101,  1045,  7078,  ...,     0,     0,     0],
        [  101,  2023,  2001,  ...,     0,     0,     0]]), tensor([4, 4, 4, 4, 1, 3, 3, 0, 4, 3, 2, 3, 0, 2, 4, 3])]
[tensor([[  101,  2044,  2667,  ...,     0,     0,     0],
        [  101,  1045,  2196,  ...,     0,     0,     0],
        [  101,  1996,  2833,  ...,    

KeyboardInterrupt: 

In [43]:
print(pooled.shape)
print(y.shape)

torch.Size([8, 768])
torch.Size([8, 512, 768])


In [45]:
print(pooled)

tensor([[-0.3662, -0.6554, -0.9980,  ..., -0.9882, -0.5634,  0.1054],
        [-0.7312, -0.6987, -0.9994,  ..., -0.9884, -0.6710,  0.4328],
        [-0.4229, -0.6139, -0.9968,  ..., -0.9809, -0.3606,  0.0971],
        ...,
        [-0.5516, -0.5865, -0.9968,  ..., -0.9810, -0.4725,  0.1771],
        [-0.5540, -0.6922, -0.9979,  ..., -0.9860, -0.6478,  0.3626],
        [-0.2911, -0.3518, -0.9795,  ..., -0.8972, -0.2740,  0.3097]],
       grad_fn=<TanhBackward0>)


In [75]:
import torch
from sklearn.model_selection import train_test_split
# convert our data into a torch dataset
#need y data from the file
x_train, x_test, y_train, y_test = train_test_split(pooled.detach(), ratings, test_size=0.15)
ratings = torch.tensor(starIDS).float()
data = torch.utils.data.TensorDataset(x_train, y_train)

# set up a data loader to get batches from our dataset
loader = torch.utils.data.DataLoader(data, batch_size = 16, shuffle = True)

In [76]:
model = torch.nn.Sequential(   
    torch.nn.Linear(768,800),
    torch.nn.Sigmoid(),
    torch.nn.Linear(800,1),
    torch.nn.Sigmoid()
)

In [77]:
scoring_function = torch.nn.MSELoss()
num_epochs = 10

optimizer = torch.optim.SGD( model.parameters(), .001 )

In [78]:
def train():
    for step in range (num_epochs):

        for batch in loader:
            optimizer.zero_grad()
            batchX = batch[0]
            #print(batchX.size())
            batchY = batch[1].unsqueeze(-1)
            #print(batchY.size())
            # do a gradient descent update of out linear model.

            # put data through the model
            preds = model(batchX)
            #print (preds.shape)
            #fixedPreds=preds[:,0]
            #print("preds: ", fixedPreds)
            # collect the error
            errors = scoring_function(preds, batchY)
            #print("errors: ", errors)

            # compute the gradient of the error with regard to the model paramenters
            errors.backward()

            # take a step in that direction
            optimizer.step()
    new_x_tensor= torch.tensor(x_test).float()
    preds= model(new_x_tensor)
    accuracy= accuracy_score(np.round(preds.detach().numpy()), y_test)
    print(accuracy)
        
            

In [79]:
train()